In [0]:
# %pip install faker

In [0]:
from faker import Faker
import pandas as pd

fake = Faker()
# Generate 1000 sales records
data = [{"order_id": fake.uuid4(), "customer_id": fake.random_int(1, 100), 
         "amount": fake.random_number(digits=3), "date": str(fake.date_this_year())} 
        for _ in range(1000)]


print(data)

In [0]:
%sql
-- Create the structure
CREATE CATALOG IF NOT EXISTS sales_prod;
CREATE SCHEMA IF NOT EXISTS sales_prod.bronze; -- Raw Ingestion
CREATE SCHEMA IF NOT EXISTS sales_prod.silver; -- Cleaned/Joined
CREATE SCHEMA IF NOT EXISTS sales_prod.gold;   -- Aggregated

-- Create a Volume to hold your landing files (JSON/CSV)
CREATE VOLUME sales_prod.bronze.landing;

In [0]:
dbutils.fs.ls ('/Volumes/sales_prod/bronze/landing/sales_data')

In [0]:
df = spark.createDataFrame(pd.DataFrame(data))
display(df)
df.write.mode("overwrite").json("/Volumes/sales_prod/bronze/landing/sales_data")

In [0]:
# # from pyspark import pipelines as dp
# from pyspark.sql.functions import col, current_timestamp

# # @dp.table
# def bronze_sales():
#     return (spark.readStream
#             .format("cloudFiles")
#             .option("cloudFiles.format", "json")
#             .option("cloudFiles.inferColumnTypes", "true")
#             .load("/Volumes/sales_prod/bronze/landing/sales_data"))

df = spark.read.json('/Volumes/sales_prod/bronze/landing/sales_data').select("order_id","customer_id","amount","date")

df.explain()

In [0]:
df.printSchema()
from pyspark.sql.functions import *
display(df.distinct())
display(df.groupBy("order_id").agg(sum("amount")))

In [0]:
# bronze_sales_df = bronze_sales()

# display(bronze_sales_df)

print(spark.version)

In [0]:
# @dp.table
# @dp.expect_or_drop("valid_amount", "amount > 0") # Data Quality Check
# @dp.expect_or_fail("has_customer_id", "customer_id IS NOT NULL")
# def silver_sales_clean():
#     return (dp.read("bronze_sales")
#             .select(
#                 col("order_id").cast("string"),
#                 col("customer_id").cast("int"),
#                 col("amount").cast("double"),
#                 col("event_time").cast("timestamp"),
#                 current_timestamp().alias("processing_time")
#             ).dropDuplicates(["order_id"]))

In [0]:
# @dp.table
# @dp.expect_or_drop("valid_amount", "amount > 0") # Data Quality Check
# @dp.expect_or_fail("has_customer_id", "customer_id IS NOT NULL")
# def silver_sales_clean():
#     return (dp.read("bronze_sales")
#             .select(
#                 col("order_id").cast("string"),
#                 col("customer_id").cast("int"),
#                 col("amount").cast("double"),
#                 col("event_time").cast("timestamp"),
#                 current_timestamp().alias("processing_time")
#             ).dropDuplicates(["order_id"]))

In [0]:
# %sql
# CREATE CONNECTION postgres_link
# TYPE POSTGRESQL
# OPTIONS (
#   host 'your-db-host.amazonaws.com',
#   port '5432',
#   user 'db_user',
#   password 'password123' -- Or use secret('scope', 'key')
# );

In [0]:
%sql
CREATE FOREIGN CATALOG my_postgres_data
USING CONNECTION postgres_link
OPTIONS (database 'production_db');

In [0]:
%sql
SELECT 
  c.customer_name, 
  o.order_amount 
FROM main.gold.customers AS c
JOIN my_postgres_data.public.orders AS o 
  ON c.id = o.customer_id;

In [0]:
spark.read.format("delta")